# Calculating PI constant using Gregory-Leibniz Series,communicating the processes using a pipe

In [3]:
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <float.h>

long double piCalc(_Bool odd){
    
    long double piDecimals = 0;
    
    if(odd){
        
        for(int i = 1; i< 1e09; i+=4){
             piDecimals += (4.0/i);
        }

        
    }else{
        
        for(int i = 3; i< 1e09; i+=4){
             piDecimals -= (4.0/i);
        }
    }
    
    return piDecimals;
}

int main(){
    
    
    int pipefd[2];
    long double buffer;
    pid_t pid;
    int p = pipe(pipefd);
    
    
    long double piChild= 0;
    long double piFather= 0;
    long double piTotal = 0;
    
    
    
    // Error validation
    if (p<0){
        perror("pipe error\n");
        exit(-1);
    }

    pid= fork();
    
     // Pid Validation
     if (pid == 0){
         buffer = piCalc(1);
         int w = write(pipefd[1], &buffer, sizeof(long double));
         
         // Write validation
         if (w==-1) perror("Write in pipe error\n");
         close(pipefd[1]);

     }else{
         
         close(pipefd[1]);
         piFather = piCalc(0);
         int r = read(pipefd[0], &buffer, sizeof(long double));
         
         piTotal = buffer + piFather;
        
         //Read Validation
         if (r<0){
            perror("read error\n");
            exit(-1);
         }
         
         printf("Pi Value: %Lf", piTotal);

     }
    exit(0);
}
    

Pi Value: 3.141593